# Iterables and Iterators


- Iterable, refers to an object that can be iterated.
- Iterator, refers to an object that implements python `Iterator Protocol`.

## Iterables Definition

In the most basic form, they are something that can be iterated over, however, this does not mean that it has to be based on indexing.
- Sequences are iterables, but so are other objects that are not sequences.
- As long as we have a way of getting a next element out of the container of objects we can call it iterable.

In [ ]:
print("---- INFINITE ITERABLE ----")
class Squares:
  def __init__(self):
    self.i = 0

  def next_(self):
    result = self.i ** 2
    self.i += 1
    return result

# In this basic form we have a Squares object that can return us the next
# square in its index each time we call the method.
sq1 = Squares()
for i in range(5):
  print(f"Square returns {sq1.next_()} on iteration: {i}")

---- INFINITE ITERABLE ----
Square returns 0 on iteration: 0
Square returns 1 on iteration: 1
Square returns 4 on iteration: 2
Square returns 9 on iteration: 3
Square returns 16 on iteration: 4


In the aproach above the object will continue to iterate and the only way to restart it is to instantiate a new one.

A way in which we could handle this is to give a length property to our iterable and raise a `StopIteration` exception.

- We will then use now the `__next__` method python uses to map to `next()`.
- We will have an `StopIteration` exception once we iterate over all elements.

In [ ]:
print("---- FINITE ITERABLE ----")
class Squares:
  def __init__(self, length):
    self.length = length
    self.i = 0
    
  def __len__(self):
    return self.length

  def __next__(self):
    if self.i >= self.length:
      raise StopIteration
    else:
      result = self.i ** 2
      self.i += 1
      return result

sq1 = Squares(length=5)
i = 0
while True:
  try:
    print(f"Square returns {next(sq1)} on iteration: {i}")
    i += 1
  except StopIteration:
    break

---- FINITE ITERABLE ----
Square returns 0 on iteration: 0
Square returns 1 on iteration: 1
Square returns 4 on iteration: 2
Square returns 9 on iteration: 3
Square returns 16 on iteration: 4


In [ ]:
print("---- RANDOM ITERABLE ----")
# to make it not an index based iterable (sequence) let's use random

import random

class RandomNumbers:
  def __init__(self, length, *, range_min=0, range_max=10):
    self.length = length
    self.range_min = range_min
    self.range_max = range_max
    self.num_requested = 0

  def __len__(self):
    return self.length

  def __next__(self):
    if self.num_requested >= self.length:
      raise StopIteration
    else:
      self.num_requested += 1
      number = random.randint(self.range_min, self.range_max)
      return number

rn1 = RandomNumbers(length=5)
i = 0
while True:
  try:
    print(f"RandomNumbers returns {next(rn1)} on iteration: {i}")
    i += 1
  except StopIteration:
    break

---- RANDOM ITERABLE ----
RandomNumbers returns 3 on iteration: 0
RandomNumbers returns 4 on iteration: 1
RandomNumbers returns 2 on iteration: 2
RandomNumbers returns 7 on iteration: 3
RandomNumbers returns 5 on iteration: 4


## Iterator Definition

While an iterable is an object that can be iterated, for us to use the built-in python functions to iterate an object mus implement the **ITERATOR PROTOCOL**.
- the `__next__` method should be created to handle the next element and raise the `StopIteration` exception.
- the `__iter__` method should be created to return the class instance (`self`) // it looks for an iterator so if returns self then self is the iterator.

In [5]:
print("---- ITERATOR ----")
class Squares:
  def __init__(self, length):
    self.length = length
    self.i = 0
    
  def __len__(self):
    return self.length

  def __next__(self):
    print("__next__ called")
    if self.i >= self.length:
      raise StopIteration
    else:
      result = self.i ** 2
      self.i += 1
      return result

  def __iter__(self):
    print("__iter__ called")
    return self

sq1 = Squares(length=5)
print('Now we can loop through our iterator with python built ins')

for i, sq in enumerate(sq1):
  print(f"iteration: {i} returns value: {sq}")

# however, the problem is that our iterator is still consumed once it is ran.
# we would have to instantiate again our class to run it again.

print("we could also use the sorted function")
sq1 = Squares(length=5)
print(f"Squares sorted in reverse is: {sorted(sq1, reverse=True)}")

---- ITERATOR ----
Now we can loop through our iterator with python built ins
__iter__ called
__next__ called
iteration: 0 returns value: 0
__next__ called
iteration: 1 returns value: 1
__next__ called
iteration: 2 returns value: 4
__next__ called
iteration: 3 returns value: 9
__next__ called
iteration: 4 returns value: 16
__next__ called
we could also use the sorted function
__iter__ called
__next__ called
__next__ called
__next__ called
__next__ called
__next__ called
__next__ called
Squares sorted in reverse is: [16, 9, 4, 1, 0]


In [7]:
# what python actualy does is to call first function iter()
sq = Squares(5)
sq_iterator = iter(sq) # it gets the iterator
print(id(sq), id(sq_iterator))

while True:
  try:
    item = next(sq)
    print(f"Value: {item}")
  except StopIteration:
    break

# We see that the prints are the same as when python runs it through a loop.

__iter__ called
140335742015312 140335742015312
__next__ called
Value: 0
__next__ called
Value: 1
__next__ called
Value: 4
__next__ called
Value: 9
__next__ called
Value: 16
__next__ called


We still need to instantiate again the iterator to loop again, this however is not useful since we create a whole new object with the same data just to iterate over it.

We can approach the problem by splitting the collection (iterable) and the iterator.
this means that:
- The collection is **Iterable**.
  - Gets created once.
- The iteror is created every time we need to start a fresh iteration.

to accomplish this we will have:
- The iterable
  - Object that implements the `__iter__` and returns an iterator (in general, a new instance of the class)
- The iterator
  - `__iter__`, returns itself, not a new instance
  - `__next__`, returns the next element

So iterators are themselves iterables but they become exhausted when they are iterated over.

In [12]:
# First the collection
class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return CitiesIterator(self)

class CitiesIterator:
  def __init__(self, cities_obj):
    self._cities_obj = cities_obj
    self._index = 0

  def __iter__(self):
    """
    This iter looks for an iterator and finds one in self, meaning it is an
    iterator.
    """
    print(" CitiesIterator __iter__ called")
    return self

  def __next__(self):
    print("  CitiesIterator __next__ called")
    if self._index >= len(self._cities_obj):
      raise StopIteration
    else:
      item = self._cities_obj._cities[self._index]
      self._index += 1
      return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

Loop 0 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 1 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 2 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIt

In [ ]:
# Nest our CitiesIterator inside our main Cities class
class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return self.CitiesIterator(self)

  class CitiesIterator:
    def __init__(self, cities_obj):
      self._cities_obj = cities_obj
      self._index = 0

    def __iter__(self):
      """
      This iter looks for an iterator and finds one in self, meaning it is an
      iterator.
      """
      print(" CitiesIterator __iter__ called")
      return self

    def __next__(self):
      print("  CitiesIterator __next__ called")
      if self._index >= len(self._cities_obj):
        raise StopIteration
      else:
        item = self._cities_obj._cities[self._index]
        self._index += 1
        return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

We can now implement a sequence and an iterator, however, once a iteration is started the `__iter__` and `__next__` methods are prefered.

Python will:
- look for the `__iter__`, if not
- look for the `__getitem__`

In [16]:
# Sequence + Iterator protocols implemented

print("---- SEQUENCE ----")

class Cities:
  def __init__(self):
    self._cities = ['Medellin', 'Cordoba', 'Cali', 'London', 'Montreal']
    self._index = 0

  def __len__(self):
    return len(self._cities)

  def __iter__(self):
    """
    this iter looks for an iterator, and finds one by instantiating a iterator
    with our object(self)
    """
    print('Cities __iter__ called')
    return self.CitiesIterator(self)

  def __getitem__(self, i):
    return self._cities[i]

  class CitiesIterator:
    def __init__(self, cities_obj):
      self._cities_obj = cities_obj
      self._index = 0

    def __iter__(self):
      """
      This iter looks for an iterator and finds one in self, meaning it is an
      iterator.
      """
      print(" CitiesIterator __iter__ called")
      return self

    def __next__(self):
      print("  CitiesIterator __next__ called")
      if self._index >= len(self._cities_obj):
        raise StopIteration
      else:
        item = self._cities_obj._cities[self._index]
        self._index += 1
        return item

# Now we can have a Cities class that loops by instantiating a CitiesIterator.
c1 = Cities()
print(f"we also have now a cities sequence c1[0] -> {c1[0]}")
for i in range(3):
  print(f"Loop {i} for the same class: {type(c1).__name__}")
  for j, city in enumerate(c1):
    print(f"\tcity:{city} in iteration: {j}")

---- SEQUENCE ----
we also have now a cities sequence c1[0] -> Medellin
Loop 0 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 1 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
  CitiesIterator __next__ called
	city:Cordoba in iteration: 1
  CitiesIterator __next__ called
	city:Cali in iteration: 2
  CitiesIterator __next__ called
	city:London in iteration: 3
  CitiesIterator __next__ called
	city:Montreal in iteration: 4
  CitiesIterator __next__ called
Loop 2 for the same class: Cities
Cities __iter__ called
  CitiesIterator __next__ called
	city:Medellin in iteration: 0
 

In [17]:
# We can verify that this is how builtins are implemented
l1 = [1,2,3,4,5]
il1 = iter(l1)
print(f"we can get the iterable for l1 -> {il1}")

for i, item in enumerate(il1):
  print(f"item: {item} at position {i} in list.")
# the iterator now gets exhausted.
try:
  next(il1)
except StopIteration:
  print("l1 iterator il1 is exhausted")

we can get the iterable for l1 -> <list_iterator object at 0x7fa27602a350>
item: 1 at position 0 in list.
item: 2 at position 1 in list.
item: 3 at position 2 in list.
item: 4 at position 3 in list.
item: 5 at position 4 in list.
l1 iterator il1 is exhausted


## Consuming Iterators Manually

In [12]:
from collections import namedtuple

cars = []

# One good way to use them is to read from a csv file
def cast(data_type, value):
  if data_type == 'DOUBLE':
    return float(value)
  elif data_type == 'INT':
    return int(value)
  else:
    return str(value)

def cast_row(data_types, data_row):
  return [cast(data_type, value) for data_type, value in zip(data_types, data_row)]

print("---- CLASSIC WAY ----")
with open('cars.csv') as file:
  row_index = 0
  for line in file:
    if row_index == 0:
      headers = line.strip('\n').split(';')
      Car = namedtuple('Car', headers)
    elif row_index == 1:
      data_types = line.strip('\n').split(';')
    else:
      data = line.strip('\n').split(';')
      data = cast_row(data_types, data)
      car = Car(*data)
      cars.append(car)
    row_index += 1

print(f"finished reading file with record count: {len(cars)}")

print("---- ITERATOR WAY ----")
cars = []
with open('cars.csv') as file:
  file_iter = iter(file)
  headers = next(file_iter).strip('\n').split(';')
  Car = namedtuple('Car', headers)
  data_types = next(file_iter).strip('\n').split(';')

  for line in file_iter:
    data = line.strip('\n').split(';')
    data = cast_row(data_types, data)
    car = Car(*data)
    cars.append(car)

print(f"finished reading file with record count: {len(cars)}")

print("---- COMPREHENSION ----")
with open('cars.csv') as file:
  file_iter = iter(file)
  headers = next(file_iter).strip('\n').split(';')
  Car = namedtuple('Car', headers)
  data_types = next(file_iter).strip('\n').split(';')

  cars_data = [cast_row(data_types, line.strip('\n').split(';')) for line in file_iter]
  car = [Car(*car) for car in cars_data]

print(f"finished reading file with record count: {len(car)}")

---- CLASSIC WAY ----
finished reading file with record count: 406
---- ITERATOR WAY ----
finished reading file with record count: 406
---- COMPREHENSION ----
finished reading file with record count: 406


## Cyclic iterators

```
1 2 3 4 5 6 7 8 ...
N S W E
```
```
1N 2S 3W 4E 5N 6S 7W 8E ...
```

In [ ]:
class CyclicIterator:
  def __init__(self, lst):
    self.lst = lst
    self.i = 0

  def __iter__(self):
    return self

  def __next__(self):
    result = self.lst[self.i % len(self.lst)] # Just for sequences.
    self.i += 1
    return result

# We can use this infinite iterator in a finite way
iter_cycl = CyclicIterator("NSWE")
for _ in range(10):
  print(next(iter_cycl))

In [23]:
# as expected Python has a cyclic iterator built in
import itertools

print("---- OUR CLASS ----")
iter_cycl = CyclicIterator('NSWE')
print([str(item_1) + item_2 for item_1, item_2 in zip(range(1, 11), iter_cycl)])

print("---- BUILT IN ----")
iter_cycl = itertools.cycle('NSWE')
print([f"{i}{next(iter_cycl)}" for i in range(1, 11)])

---- OUR CLASS ----
['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']
---- BUILT IN ----
['1N', '2S', '3W', '4E', '5N', '6S', '7W', '8E', '9N', '10S']


In [25]:
# this one receives any iterable not just sequences
class CyclicIterator2:
  def __init__(self, lst):
    self.lst = lst
    self.iterator = iter(self.lst)

  def __iter__(self):
    return self

  def __next__(self):
    try:
      result = next(self.iterator)
    except StopIteration:
      self.iterator = iter(self.lst)
      result = next(self.iterator)
    return result

# We can use this infinite iterator in a finite way
iter_cycl = CyclicIterator2("NSWE")
for _ in range(10):
  print(next(iter_cycl))

N
S
W
E
N
S
W
E
N
S


## Lazy Iterables

In [27]:
import math

class Circle:
  def __init__(self, r):
    self.radius = r

  def __repr__(self):
    return f"Circle(r={self.radius})"

  @property
  def radius(self):
    return self._radius

  @radius.setter
  def radius(self, r):
    self._radius = r
    self.area = math.pi * (r ** 2)

c1 = Circle(5)
print(f"Circle: {c1} with area: {c1.area}")

# Here we have to calculate the area each time we set a new radius, let's fix that

Circle: Circle(r=5) with area: 78.53981633974483


In [30]:
class Circle:
  def __init__(self, r):
    self.radius = r
    self._area = None

  def __repr__(self):
    return f"Circle(r={self.radius})"

  @property
  def radius(self):
    return self._radius

  @radius.setter
  def radius(self, r):
    self._radius = r
    self._area = None

  @property
  def area(self):
    if self._area is None:
      print('Calculating Area')
      self._area = math.pi * (self._radius ** 2)
    return self._area

c1 = Circle(5)
print(f"Circle: {c1} with area: {c1.area}")
# No calculation of area again (no print statement)
print(f"Circle: {c1} with area: {c1.area}")
c1.radius = 3
print(f"Circle: {c1} with area: {c1.area}")

# this is the lazy evaluation for a property!!

Calculating Area
Circle: Circle(r=5) with area: 78.53981633974483
Circle: Circle(r=5) with area: 78.53981633974483
Calculating Area
Circle: Circle(r=3) with area: 28.274333882308138


In [ ]:
# Example 2
class Factorials:

  def __iter__(self):
    return self.FactIter()

  class FactIter:
    def __init__(self):
      self.i = 0

    def __iter__(self):
      return self

    def __next__(self):
      result = math.factorial(self.i)
      self.i += 1
      return result

f1 = Factorials()
for fact in range(20):
  print(fact)

## Built-in iterables and iterators

Some of the python Built-ins return either an Iterator or an Iterable.
- Iterable, if the `__iter__` returns an iterator
- Iterator if the `__iter__` returns self.

In [50]:
print("---- RANGE/ITERABLE ----")
r = range(10)
print(f"range: {r} of type: {type(r)} has __iter__ method: {'__iter__' in dir(r)}")
print(f"range: {r} of type: {type(r)} has __next__ method: {'__next__' in dir(r)}")
print(f"range: {r} of type: {type(r)} has __iter__ mehtod return self: {iter(r) is r}") 
r_iterator = iter(r)
print(f"we can create an iterator: {r_iterator} from range: {r}")

print("---- ZIP/ITERATOR ----")
# zip uses lazy evaluation, the only way to get the items is by exhausting the 
# iterator.
z = zip([1,2,3], 'abc')
print(f"zip: {z} of type: {type(z)} has __iter__ method: {'__iter__' in dir(z)}")
print(f"zip: {z} of type: {type(z)} has __next__ method: {'__next__' in dir(z)}")
print(f"zip: {z} of type: {type(z)} has __iter__ mehtod return self: {iter(z) is z}")  


print("---- OPEN/ITERATOR ----")
# open returns an iterator on an open file, and it uses lazy evaluation to return
# the contents of the file as they are needed.
with open("cars.csv") as file:
  print(f"open file: {file} of type: {type(file)} has __iter__ method: {'__iter__' in dir(file)}")
  print(f"open file: {file} of type: {type(file)} has __next__ method: {'__next__' in dir(file)}")
  print(f"open file: {file} of type: {type(file)} has __iter__ mehtod return self: {iter(file) is file}")  

print("---- ENUMERATE/ITERATOR ----")
e = enumerate('Python Rocks!!')
print(f"enumerate: {e} of type: {type(e)} has __iter__ mehtod: {'__iter__' in dir(e)}")
print(f"enumerate: {e} of type: {type(e)} has __next__ mehtod: {'__next__' in dir(e)}")
print(f"enumerate: {e} of type: {type(e)} has __iter__ mehtod return self: {iter(e) is e}")

---- RANGE/ITERABLE ----
range: range(0, 10) of type: <class 'range'> has __iter__ method: True
range: range(0, 10) of type: <class 'range'> has __next__ method: False
range: range(0, 10) of type: <class 'range'> has __iter__ mehtod return self: False
we can create an iterator: <range_iterator object at 0x7f5cc270c150> from range: range(0, 10)
---- ZIP/ITERATOR ----
zip: <zip object at 0x7f5cc290eb90> of type: <class 'zip'> has __iter__ method: True
zip: <zip object at 0x7f5cc290eb90> of type: <class 'zip'> has __next__ method: True
zip: <zip object at 0x7f5cc290eb90> of type: <class 'zip'> has __iter__ mehtod return self: True
---- OPEN/ITERATOR ----
open file: <_io.TextIOWrapper name='cars.csv' mode='r' encoding='UTF-8'> of type: <class '_io.TextIOWrapper'> has __iter__ method: True
open file: <_io.TextIOWrapper name='cars.csv' mode='r' encoding='UTF-8'> of type: <class '_io.TextIOWrapper'> has __next__ method: True
open file: <_io.TextIOWrapper name='cars.csv' mode='r' encoding='UTF

## Sorting Iterables

In [52]:
import random

class RandomInts:
  def __init__(self, length, *, seed=0, lower=0, upper=10):
    self.length = length
    self.seed = seed
    self.lower = lower
    self.upper = upper

  def __len__(self):
    return self.length

  def __iter__(self):
    return self.RandomIterator(self.length, seed=self.seed, lower=self.lower, upper=self.upper)

  class RandomIterator:
    def __init__(self, length, *, seed, lower, upper):
      self.length = length
      self.lower = lower
      self.upper = upper
      self.num_requests = 0
      random.seed(seed)

    def __iter__(self):
      return self

    def __next__(self):
      if self.num_requests >= self.length:
        raise StopIteration
      else:
        result = random.randint(self.lower, self.upper)
        self.num_requests += 1
        return result

ri = RandomInts(10)
print(f"the sorted items for RandomInts(10): {ri} are: {sorted(ri)}")

the sorted items for RandomInts(10): <__main__.RandomInts object at 0x7f5cc27e0dd0> are: [0, 4, 4, 5, 6, 6, 6, 7, 7, 8]


## The Iter Function

when we call the `iter()` python looks for in order:
- `__iter__` method, if found it calls it.
- `__getitem__` method, if found it implements an iterator based on it and returns it.

if None of those are found an `TypeError` is raised.

In [55]:
# if python does not find the __iter__ method it can actually implement
# the following iterator if the object is a sequence (implements __getitem__)
class SequenceIterator:
  def __init__(self, seq):
    self._sequence = seq
    self.i = 0

  def __iter__(self):
    return self

  def __next__(self):
    if self.i >= len(self._sequence):
      raise StopIteration
    else:
      result = self._sequence[i]
      self.i += 1
      return result
      
class SimpleIter:
  def __iter__(self):
    return 'Nope' # Implements the iter, but it is not an iterator

def is_iterable(obj):
  try:
    iter(obj)
    return True
  except TypeError:
    return False

print(f"the SimpleIter class has the __iter__ so is it an iterator: {is_iterable(SimpleIter())}")

the SimpleIter class has the __iter__ so is it an iterator: False


There is also a second form to a call to `iter` and that is: `iter(callable, sentinel)`.
- A Callable object that returns the value.
- A sentinel value expected to raise a `StopIteration` if returned by the callable. (if never returned then we basically have an infinite iterator).

In [23]:
# let's create a callable iterator that takes a callable object and a 
# sentinel value.
print("---- CALLABLE ITERATOR CLASS ----")
# callable counter
def counter():
  i = 0

  def inc():
    nonlocal i
    i += 1
    return i
  return inc

# Generic callable iterator
class CallableIterator:
  def __init__(self, callable_, sentinel):
    self.callable = callable_
    self.sentinel = sentinel
    self.is_consumed = False

  def __iter__(self):
    return self

  def __next__(self):
    if self.is_consumed:
      raise StopIteration
    else:
      result = self.callable()
      if result == self.sentinel:
        self.is_consumed = True
        raise StopIteration
      else:
        return result

cnt = counter()
cnt_iterator = CallableIterator(cnt, 10)

for i in cnt_iterator:
  print(i)

---- CALLABLE ITERATOR CLASS ----
1
2
3
4
5
6
7
8
9


In [24]:
# Now we can use the iter() function to do what we did above
print("---- ITER FUNCTION ----")
cnt = counter()
cnt_iterator = iter(cnt, 10)

for i in cnt_iterator:
  print(i)

print("---- RANDOM NUMS ----")
import random
random.seed(0)
random_iter = iter(lambda : random.randint(0, 10), 8)

for num in random_iter:
  print(num)

---- ITER FUNCTION ----
1
2
3
4
5
6
7
8
9
---- RANDOM NUMS ----
6
6
0
4


## Delegating Iterators

If we are creating classes and the attribute or object which we would like to iterate over is already iterable we can delegate it to the class.

In [28]:
from collections import namedtuple

Person = namedtuple('Person', 'first last')

class PersonNames:
  def __init__(self, persons):
    try:
      self._persons = [person.first.capitalize() + " " + person.last.capitalize()
                      for person in persons]
    except (TypeError, AttributeError):
      self._persons = []

  def __iter__(self):
    return iter(self._persons) # delegate iter to list

persons = [Person('JAMES', 'hetfield'), Person('goku', 'kakarot'), Person('John', 'doe')]
person_names = PersonNames(persons)

for person in person_names:
  print(person)

James Hetfield
Goku Kakarot
John Doe


## Reversed

`reversed` returns an iterator which can be used to iterate in reverse order, This does not create a copy of the object.

It works in a similar fashion to the `iter` function, it will first look for.
- `__reversed__` method, it should return the iterator object. if not found it will then continue.
- `__getitem__` and `__len__` to create an iterator (sequence) in reverse order.

In [42]:
from collections import namedtuple

_SUITS = ('Spades', 'Hearts', 'Diamonds', 'Clubs')
_RANKS = tuple(range(2, 11)) + tuple('JQKA')

Card = namedtuple('Card', 'rank suit')

class CardDeck:
  def __init__(self):
    self.length = len(_SUITS) * len(_RANKS)

  def __len__(self):
    return self.length

  def __iter__(self):
    return self.CardDeckIterator(self.length)

  def __reversed__(self):
    return self.CardDeckIterator(self.length, reverse=True) 
  
  class CardDeckIterator:
    def __init__(self, length, reverse=False):
      self.length = length
      self.reverse = reverse
      self.i = 0

    def __iter__(self):
      return self
    
    def __next__(self):
      if self.i >= self.length:
        raise StopIteration
      else:
        if self.reverse:
          index = self.length - 1 - self.i
        else:
          index = self.i
        suit = _SUITS[index // len(_RANKS)]
        rank = _RANKS[index % len(_RANKS)]
        self.i += 1
        return Card(rank, suit)


card_deck = CardDeck()
print(type(reversed(card_deck)))
for card in reversed(card_deck):
  print(card)

<class 'str'>
h
o
l
i


In [39]:
# Reversed in sequences

print("---- WITHOUT REVERSED ----")

class Squares:
  def __init__(self, length):
    self.squares = [i ** 2 for i in range(length)]

  def __len__(self):
    return len(self.squares)

  def __getitem__(self, s):
    return self.squares[s]

squares = Squares(10)

for num in reversed(squares):
  print(num)

# As we can see reversed found __getitem__ and __len__ instead of
# the __reversed__ function and it used it to create the reversed iterator

print("---- WITH REVERSED ----")

class Squares:
  def __init__(self, length):
    self.squares = [i ** 2 for i in range(length)]

  def __len__(self):
    return len(self.squares)

  def __getitem__(self, s):
    return self.squares[s]
  
  def __reversed__(self):
    return 'Oh no'
  
squares = Squares(10)

for num in reversed(squares):
  print(num)

---- WITHOUT REVERSED ----
81
64
49
36
25
16
9
4
1
0
---- WITH REVERSED ----
O
h
 
n
o


## Caveats

- Using Iterators as Function Arguments.
  - Remember that iterators get consumed so if we chain functions which arguments are iterator they will only work on the first pass or the first place the iterator is called.

In [44]:
# To avoid this we can check if the input is an iterator and
# raise an exception.

l1 = iter([1, 2, 3, 4])

print(f"l1 is an iterator {l1}")
print(f"iter(l1) is l1 -> {iter(l1) is l1}")

# by checking if iter returns self we can verify an object is an iterator.

l1 is an iterator <list_iterator object at 0x7f65c6f7fe50>
iter(l1) is l1 -> True
